In [1]:
import numpy as np
import time
from copy import deepcopy as dc
import xarray as xr
from scipy.optimize import root

In [2]:
def liquid(sulfur):
    return 1811 - (550/0.31)*sulfur

def rho_mix(sulfur):
    x = (sulfur/32.07) / (sulfur/32.07 + (1-sulfur)/55.85) #atom fraction sulfur from wt%
    return 6950-5176*x-3108*x**2

def get_newS(sulfur, new_rho):
    return new_rho - rho_mix(sulfur)

def body_size(R, M, sulfur, mantle):
    return M - 4/3*np.pi*rho_mix(sulfur)*(R-mantle)**3 - 4/3*np.pi*rho_rock*(R**3-(R-mantle)**3)

In [3]:
k0_liq = 40 #[W/m K] thermal conductivity of Fe
k0_solid = 30 #[W/m K] thermal conductivity of Fe
rho_Fe = 7500. #[kg/m^3] density of kamacite (like Elkins-Tanton+ 2019 used)
Cp = 835 #[J/kg K] Heat capacity (of Iron)
k0_FeS = 4.6
k0_eu = k0_solid * (1-0.685) + k0_FeS * 0.685
Cp_FeS = 600
Cp_eu = Cp * (1-0.685) + Cp_FeS * 0.685
Fe_perV = (31.5*rho_Fe)/(68.5*rho_mix(0.31)+31.5*rho_Fe)
rho_eu = rho_Fe * (1-0.685) + rho_mix(0.31) * 0.685
alpha = k0_liq /(rho_Fe*Cp) #[m^2/s] Thermal diffusivity of liquid Fe
expans = 9.2e-5 #[1/K] Thermal expansivity of liquid Fe
L_Fe = 2.56e5 #[J/kg] latent heat of pure iron
L_eu = 1.33e5 #[J/kg] latent heat of eutectic mixture

# k0_rock = 4 #[W/m K]
k0_rock = 3.0 #[W/m K]
rho_rock = 3300 #[kg/m^3]
Cp_rock = 1000 

G = 6.67e-11 #[m^3/kg– s^2] Big G

exp_Fe = 9.2e-5
exp_rock = 2e-5

V_liq = 7.956
V_gamma = 7.533
V_alpha = 7.578

# Thermal Evolution Models

In [4]:
def inward(rs, IC, tmax, crust_inds, fluid_inds, t_acc=0, sulf=0, phi_mant=0, full=True, eutectic=False):
#     t_acc is initial formation time (in Myr)
#     sulf is initial (uniform) sulfur content of core (wt fraction)

    if not (np.shape(rs) == np.shape(IC)):
        print('Error: Initial Conditions Shape Mismatch')
        return
    
    print('Initial Sulfur Content:', sulf*100, 'wt%')
    
    Tinit = IC
    Ts = [Tinit]
    time = [t_acc]
    sulfur = [sulf]
    initial_solid = np.full(np.shape(IC), 0.)
    initial_solid[crust_inds] = 1.
    fs = [initial_solid]
    
    alpha = np.full(np.shape(Tinit), k0_liq /(rho_Fe*Cp)) #assume all iron is liquid init
    alpha[crust_inds] = k0_rock * (1-phi_mant) / (rho_rock*Cp_rock)
    alphas = [alpha]
    
    if sulf >= 0.31:
        eutectic = True
    
    liquidus = liquid(sulf)
    
    limit = np.nanmin(np.diff(rs)**2/(2*alpha[:-1]))
    dt = limit/50
    M = 1e8 #tmax/100
    
    
    t = dt + (t_acc*1e6*365*86400)
   
    reached = 0
    for k in range(1,int(M)+1): #timesteps
        a = rs[0]
        drs = np.diff(rs)
        Tk = Ts[k-1]
        Tnext = np.zeros(np.shape(Tk))
        next_solid = dc(fs[k-1])
        next_alpha = dc(alpha)
        
        Sk = np.zeros(np.shape(rs))

        limits = drs**2 / (2*alpha[:-1])
        if np.nanmin(limits) < dt:
            print('Unstable timestep')
            return np.asarray(Ts), np.asarray(time)
        
        icb_ind = np.nanmax(fluid_inds) + 1 
        R_ic = rs[icb_ind]
        ocb_ind = np.nanmin(crust_inds)
        R_oc = rs[ocb_ind]
        
        q_ic = -k0_liq*(Tk[icb_ind] - Tk[icb_ind-1])/ drs[icb_ind] #negative
        M_ic = 4/3*np.pi*R_ic**3*rho_mix(sulf)
        

        # the first time we start solidification
        if Tk[icb_ind-1] <= liquidus and R_ic == R_oc:
            print('solidification start {} Myr'.format(t/(1e6*365*86400)))
            fluid_inds = dc(fluid_inds[:-1])
            icb_ind = np.nanmax(fluid_inds) + 1 
            R_ic = rs[icb_ind]
            
            q_ic = -k0_liq*(Tk[icb_ind] - Tk[icb_ind-1])/ drs[icb_ind] #negative
            M_ic = 4/3*np.pi*R_ic**3*rho_mix(sulf)
            
        
        #hotter than liquidus and no solid iron bit to conduct through
        if Tk[icb_ind-1] > liquidus and R_ic == R_oc:
            temp_change = q_ic*(4*np.pi*R_ic**2)*dt / (M_ic*Cp)
            T_ic = Tk[icb_ind-1] - temp_change #last top temp - adiabatic flux*dr/k_liq

            Tnext[fluid_inds] = T_ic
            
            q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind]
            new_inds = dc(fluid_inds)
            
        #hotter than liquidus and solid iron bit to conduct through
        elif Tk[icb_ind-1] > liquidus:
            q_ic = -k0_liq*(Tk[icb_ind+1] - Tk[icb_ind])/ drs[icb_ind] #negative
            temp_change = q_ic*(4*np.pi*R_ic**2)*dt / (M_ic*Cp)
            T_ic = Tk[icb_ind-1] - temp_change #last top temp - adiabatic flux*dr/k_liq

            Tnext[fluid_inds] = T_ic
            Tnext[icb_ind] = T_ic
            
            q_oc = k0_solid*(Tk[icb_ind+1] - Tk[icb_ind])/ drs[icb_ind]
            Tk_oc = Tk[icb_ind+1:ocb_ind+1]
            rs_oc = rs[icb_ind+1:ocb_ind+1]
            drs_oc = np.diff(rs_oc)
            alpha_oc = alpha[icb_ind+1:ocb_ind+1]
            Sk_oc = Sk[icb_ind+1:ocb_ind+1]


            Tnext[icb_ind+1] = Tk_oc[0] + 2*alpha_oc[0] * (dt / drs_oc[0]**2) * (Tk_oc[1] - Tk_oc[0] - \
                            q_oc*drs_oc[0]) + 2 * alpha_oc[0] * (dt / R_ic) * q_oc + Sk_oc[0] * dt   


            Tnext[icb_ind+2:ocb_ind] = Tk_oc[1:-1] + alpha_oc[1:-1]*(dt/drs_oc[1:]**2) * (Tk_oc[2:]-2*Tk_oc[1:-1] + \
                         Tk_oc[0:-2]) + alpha_oc[1:-1] * (dt/(rs_oc[1:-1]*drs_oc[1:])) * (Tk_oc[2:] - Tk_oc[0:-2]) + \
                         dt * Sk_oc[1:-1]

            q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs_oc[-1]

            new_inds = dc(fluid_inds)
             
        else:
            Tnext[fluid_inds] = liquidus #keep core at liquidus
            f_solid = next_solid[icb_ind]
            
            Tnext[icb_ind] = liquidus
            
            if eutectic == False:
                df_solid = -3/(L_Fe*rho_Fe*((R_ic+drs[icb_ind])**3-R_ic**3))*(k*dt/drs[icb_ind])*(R_ic+drs[icb_ind])**2* \
                            (Tk[icb_ind+1]-liquidus)
            else:
                df_solid = -3/(L_eu*rho_eu*((R_ic+drs[icb_ind])**3-R_ic**3))*(k*dt/drs[icb_ind])* \
                            (R_ic+drs[icb_ind])**2*(Tk[icb_ind+1]-liquidus)

            f_solid += df_solid
            next_solid[icb_ind] += df_solid
        
            # cell is fully solidified            
            if f_solid >= 1 and eutectic == False:
                new_inds = dc(fluid_inds[:-1])
                next_solid[icb_ind] = 1.
                next_alpha[icb_ind] = k0_solid / (rho_Fe*Cp)
            elif f_solid >= 1 and eutectic == True:
                new_inds = dc(fluid_inds[:-1])
                next_solid[icb_ind] = 1.
                next_alpha[icb_ind] = k0_eu / (rho_eu*Cp_eu)
            else:
                new_inds = dc(fluid_inds)           
            
            if icb_ind + 1 == ocb_ind:
                #solid outer core hasn't formed yet
                q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind]
                
            else:
                q_oc = k0_solid*(Tk[icb_ind+1] - Tk[icb_ind])/ drs[icb_ind]
                Tk_oc = Tk[icb_ind+1:ocb_ind+1]
                rs_oc = rs[icb_ind+1:ocb_ind+1]
                drs_oc = np.diff(rs_oc)
                alpha_oc = alpha[icb_ind+1:ocb_ind+1]
                Sk_oc = Sk[icb_ind+1:ocb_ind+1]
                

                Tnext[icb_ind+1] = Tk_oc[0] + 2*alpha_oc[0] * (dt / drs_oc[0]**2) * (Tk_oc[1] - Tk_oc[0] - \
                                q_oc*drs_oc[0]) + 2 * alpha_oc[0] * (dt / R_ic) * q_oc + Sk_oc[0] * dt   
                

                Tnext[icb_ind+2:ocb_ind] = Tk_oc[1:-1] + alpha_oc[1:-1]*(dt/drs_oc[1:]**2) * (Tk_oc[2:]-2*Tk_oc[1:-1] + \
                             Tk_oc[0:-2]) + alpha_oc[1:-1] * (dt/(rs_oc[1:-1]*drs_oc[1:])) * (Tk_oc[2:] - Tk_oc[0:-2]) + \
                             dt * Sk_oc[1:-1]
                
                q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs_oc[-1]
        
        # Set Up Stuff for Crust
        Tk_crust = Tk[crust_inds]
        rs_crust = rs[crust_inds]
        drs_crust = np.diff(rs_crust)
        alpha_crust = alpha[crust_inds]
        Sk_crust = Sk[crust_inds]
        
        q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind]
        
        if len(crust_inds) > 1:
            Tnext[crust_inds[0]] = Tk_crust[0] + 2*alpha_crust[0] * (dt / drs_crust[0]**2) * \
                    (Tk_crust[1] - Tk_crust[0] - q_crust*drs_crust[0]) + 2 * alpha_crust[0] * (dt / rs_crust[0]) * \
                    q_crust + Sk_crust[0] * dt

            Tnext[crust_inds[1:-1]] = Tk_crust[1:-1] + alpha_crust[1:-1]*(dt/drs_crust[1:]**2) * \
                            (Tk_crust[2:]-2*Tk_crust[1:-1] + Tk_crust[0:-2]) + alpha_crust[1:-1] * \
                            (dt/(rs_crust[1:-1]*drs_crust[1:])) * (Tk_crust[2:] - Tk_crust[0:-2]) + \
                            dt * Sk_crust[1:-1]

        #constant surface temp
        Tnext[-1] = 137
        
    
        ## UPDATE EVERYTHING FOR SOLIDIFICATION  
        if len(new_inds) == 0:
            print('fully solidified {} Myr'.format(t/(1e6*365*86400)))
            Ts.append(Tnext)
            time.append(t)
            sulfur.append(sulf)
            next_solid[:] = 1. #the last cell is only ~99% solid
            fs.append(next_solid)
            if eutectic == False:
                next_alpha[0] = k0_solid / (rho_Fe*Cp)
            else:
                next_alpha[0] = k0_eu / (rho_eu*Cp_eu)
            alphas.append(next_alpha)
            reached = dc(k)
            t += dt
            if full:
                break
            else:
                return np.asarray(Ts), np.asarray(time), np.asarray(sulfur), np.asarray(fs), np.asarray(alphas)
        
            
        if sulf == 0:
            new_sulf = 0
        elif sulf < 0.31:
            M0 = 4/3*np.pi*rs[icb_ind]**3*rho_mix(sulf)
            dM = 4/3*np.pi*rho_Fe*(rs[icb_ind]**3-rs[np.nanmax(new_inds)+1]**3)
            rnew = rs[np.nanmax(new_inds)+1]
            if dM == 0.:
                new_sulf = dc(sulf)
            else:
                sol = root(get_newS, sulf, args=((M0-dM)/(4/3*np.pi*rnew**3)))
                new_sulf = sol.x[0]
        else:
            new_sulf = 0.31
        
        if new_sulf > sulf:
            if dM == 0.:
                print('uh oh', new_sulf - sulf)
            if new_sulf >= 0.31:
                print('eutectic at {} Myr'.format(t/(1e6*365*86400)))
        
        if new_sulf >= 0.31:
            new_sulf = 0.31
            eutectic = True
            
        sulf = dc(new_sulf)
        fluid_inds = dc(new_inds)
        alpha = dc(next_alpha)
        liquidus = liquid(sulf)

        Ts.append(Tnext)
        time.append(t)
        sulfur.append(sulf)
        fs.append(next_solid)
        alphas.append(alpha)

        if t > tmax*3.154e+7:
            print('Reached tmax')
            return np.asarray(Ts), np.asarray(time), np.asarray(sulfur), np.asarray(fs), np.asarray(alphas)

        #change to lower resolution if possible
        limits = np.diff(rs)**2 / (2*alpha[:-1])
        dt = np.nanmin(limits) / 50 #take 1/50th of stability limit as timestep

        t += dt
    
#     NEW LOOP FOR FULL CONDUCTION
    for k in range(reached+1,int(M)+1): #timesteps
        a = rs[0]
        drs = np.diff(rs)
        Tk = Ts[k-1]
        alpha = dc(alphas[k-1])
        Tnext = np.zeros(np.shape(Tk))
        next_solid = dc(fs[k-1])

        ocb_ind = np.nanmin(crust_inds)
        R_oc = rs[ocb_ind]
            
        q_oc = -k0_solid*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind] #negative
        q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind]

        Tk_ic = Tk[:ocb_ind+1]
        rs_ic = rs[:ocb_ind+1]
        drs_ic = np.diff(rs_ic)
        alpha_ic = alpha[:ocb_ind+1]
        Sk_ic = Sk[:ocb_ind+1]

        #Zero Flux BC
        Tnext[0] = Tk_ic[0] + 2*alpha_ic[0] * (dt / drs_ic[0]**2) * (Tk_ic[1] - Tk_ic[0]) + Sk_ic[0] * dt   


        Tnext[1:ocb_ind] = Tk_ic[1:-1]+alpha_ic[1:-1]*(dt/drs_ic[1:]**2)*(Tk_ic[2:]-2*Tk_ic[1:-1] + \
                     Tk_ic[0:-2]) + alpha_ic[1:-1]*(dt/(rs_ic[1:-1]*drs_ic[1:]))*(Tk_ic[2:] - Tk_ic[0:-2]) + \
                     dt * Sk_ic[1:-1]
        
        # Set Up Stuff for Crust
        Tk_crust = Tk[crust_inds]
        rs_crust = rs[crust_inds]
        drs_crust = np.diff(rs_crust)
        alpha_crust = alpha[crust_inds]
        Sk_crust = Sk[crust_inds]
               
        
        if len(crust_inds) > 1:
            Tnext[crust_inds[0]] = Tk_crust[0] + 2*alpha_crust[0] * (dt / drs_crust[0]**2) * \
                    (Tk_crust[1] - Tk_crust[0] - q_crust*drs_crust[0]) + 2 * alpha_crust[0] * (dt / R_oc) * \
                    q_crust + Sk_crust[0] * dt
        

            Tnext[crust_inds[1:-1]] = Tk_crust[1:-1] + alpha_crust[1:-1]*(dt/drs_crust[1:]**2) * \
                            (Tk_crust[2:]-2*Tk_crust[1:-1] + Tk_crust[0:-2]) + alpha_crust[1:-1] * \
                            (dt/(rs_crust[1:-1]*drs_crust[1:])) * (Tk_crust[2:] - Tk_crust[0:-2]) + \
                            dt * Sk_crust[1:-1]

        #constant surface temp
        Tnext[-1] = 137
        
        Ts.append(Tnext)
        time.append(t)
        sulfur.append(sulf)
        fs.append(next_solid)
        alphas.append(alpha)

        if t > tmax*3.154e+7:
            print('Reached tmax')
            return np.asarray(Ts), np.asarray(time), np.asarray(sulfur), np.asarray(fs), np.asarray(alphas)

        #change to lower resolution if possible
        limits = np.diff(rs)**2 / (2*alpha[:-1])
        dt = np.nanmin(limits) / 5 #take 1/5th of stability limit as timestep

        t += dt
    
    return np.asarray(Ts), np.asarray(time), np.asarray(sulfur), np.asarray(fs), np.asarray(alphas)

In [5]:
def outward(rs, IC, tmax, crust_inds, fluid_inds, t_acc=0, sulf=0, phi_mant=0, full=True, eutectic=False):
#     t_acc is initial formation time (in Myr)
#     sulf is initial (uniform) sulfur content of core (wt fraction)

    if not (np.shape(rs) == np.shape(IC)):
        print('Error: Initial Conditions Shape Mismatch')
        return
    
    print('Initial Sulfur Content:', sulf*100, 'wt%')
    
    Tinit = IC
    Ts = [Tinit]
    time = [t_acc]
    sulfur = [sulf]
    initial_solid = np.full(np.shape(IC), 0.)
    initial_solid[crust_inds] = 1.
    fs = [initial_solid]
    
    alpha = np.full(np.shape(Tinit), k0_liq /(rho_Fe*Cp)) #assume all iron is liquid init
    alpha[crust_inds] = k0_rock * (1-phi_mant) / (rho_rock*Cp_rock)  
    alphas = [alpha]
    
    if sulf >= 0.31:
        eutectic = True
    
    liquidus = liquid(sulf)
    
    limit = np.nanmin(np.diff(rs)**2/(2*alpha[:-1]))
    dt = limit/50
    M = 1e8 #tmax/100
    
    t = dt + (t_acc*1e6*365*86400)
   
    reached = 0.
    for k in range(1,int(M)+1): #timesteps
        a = rs[0]
        drs = np.diff(rs)
        Tk = Ts[k-1]
        Tnext = np.zeros(np.shape(Tk))
        next_solid = dc(fs[k-1])
        next_alpha = dc(alpha)
        
        Sk = np.zeros(np.shape(rs))

        limits = drs**2 / (2*alpha[:-1])
        if np.nanmin(limits) < dt:
            print('Unstable timestep')
            return np.asarray(Ts), np.asarray(time)
        
        icb_ind = np.nanmin(fluid_inds)
        R_ic = rs[icb_ind]
        ocb_ind = np.nanmin(crust_inds)
        R_oc = rs[ocb_ind]
        sol_ind = ocb_ind - 1
            
        q_oc = -k0_liq*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind] #negative
        M_oc = 4/3*np.pi*(R_oc**3 - R_ic**3)*rho_mix(sulf)
        
        # the first time we start solidification
        if Tk[ocb_ind-1] <= liquidus and ocb_ind == np.nanmax(fluid_inds)+1:
            print('solidification start {} Myr'.format(t/(1e6*365*86400)))
            fluid_inds = dc(fluid_inds[:-1])
        
        if Tk[ocb_ind-1] > liquidus:
            ## Fluid Inner Core - isothermal (adiabat is only like 2K difference top to bottom)
            q_oc = -k0_liq*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind]
            if R_ic > 0.:
                q_ic = k0_liq*(Tk[icb_ind] - Tk[icb_ind-1]) / drs[icb_ind]
            else:
                q_ic = 0.
            temp_change = (q_ic+q_oc)*(4*np.pi*R_oc**2)*dt / (M_oc*Cp)
#             print(temp_change)
            T_oc = Tk[ocb_ind-1] - temp_change #last top temp - adiabatic flux*dr/k_liq

            Tnext[fluid_inds] = T_oc 
            Tnext[sol_ind] = T_oc
            
            q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind] #no solid iron bit to conduct through
            new_inds = dc(fluid_inds)
            if R_ic > 0.: #theres a conductive inner core
                q_ic = -k0_solid*(Tk[icb_ind] - Tk[icb_ind-1]) / drs[icb_ind] #flux out of ic (reverse for liquid core)
                
                Tk_ic = Tk[:icb_ind+1]
                rs_ic = rs[:icb_ind+1]
                drs_ic = np.diff(rs_ic)
                alpha_ic = alpha[:icb_ind+1]
                Sk_ic = Sk[:icb_ind+1]
                
                #Zero Flux BC
                Tnext[0] = Tk_ic[0] + 2*alpha_ic[0] * (dt / drs_ic[0]**2) * (Tk_ic[1] - Tk_ic[0]) + Sk_ic[0] * dt   


                Tnext[1:icb_ind] = Tk_ic[1:-1]+alpha_ic[1:-1]*(dt/drs_ic[1:]**2)*(Tk_ic[2:]-2*Tk_ic[1:-1] + \
                             Tk_ic[0:-2]) + alpha_ic[1:-1]*(dt/(rs_ic[1:-1]*drs_ic[1:]))*(Tk_ic[2:] - Tk_ic[0:-2]) + \
                             dt * Sk_ic[1:-1]
    
                Tnext[icb_ind] = T_oc
        
        else:
            Tnext[fluid_inds] = liquidus #keep core at liquidus
            sol_ind = ocb_ind - 1
            f_solid = next_solid[sol_ind]
            
            Tnext[sol_ind] = liquidus
            V_layer = 4/3 * np.pi * ((R_oc)**3 - (R_oc-drs[sol_ind])**3)
            
            if eutectic == False:
                df_solid = -3/(L_Fe*rho_Fe*(R_oc**3-(R_oc-drs[sol_ind])**3))*(k*dt/drs[sol_ind])*R_oc**2* \
                                (Tk[ocb_ind]-liquidus)
            else:
                df_solid = -3/(L_eu*rho_eu*(R_oc**3-(R_oc-drs[sol_ind])**3))*(k*dt/drs[sol_ind])*R_oc**2* \
                                (Tk[ocb_ind]-liquidus)                

            f_solid += df_solid
            next_solid[sol_ind] += df_solid
        
            # cell is fully solidified           
            if f_solid >= 1 and eutectic == False:
                new_icb = np.cbrt(3/(4*np.pi)*V_layer+R_ic**3)
                closest = (np.abs(rs - new_icb)).argmin()
                err = (rs[closest]-new_icb)/new_icb*100
                if err > 1:
                    print('Error in Volume over 1%', err) #print how close my dr is allowing
                 
                new_inds = np.arange(closest,np.nanmax(fluid_inds)+1, dtype=int)
                next_solid[icb_ind:closest] = 1.
                next_solid[sol_ind] = 0.
                next_alpha[icb_ind:closest] = k0_solid / (rho_Fe*Cp)
            elif f_solid >= 1 and eutectic == True:
                new_icb = np.cbrt(3/(4*np.pi)*V_layer+R_ic**3)
                closest = (np.abs(rs - new_icb)).argmin()
                err = (rs[closest]-new_icb)/new_icb*100
                if err > 1:
                    print('Error in Volume over 1%', err) #print how close my dr is allowing
                 
                new_inds = np.arange(closest,np.nanmax(fluid_inds)+1, dtype=int)
                next_solid[icb_ind:closest] = 1.
                next_solid[sol_ind] = 0.
                next_alpha[icb_ind:closest] = k0_eu / (rho_eu*Cp_eu)
            else:
                new_inds = dc(fluid_inds)
                
            
            q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind] 
            
            if R_ic > 0.: #theres a conductive inner core
                q_ic = -k0_solid*(Tk[icb_ind] - Tk[icb_ind-1]) / drs[icb_ind] #flux out of ic (reverse for liquid core)
                
                Tk_ic = Tk[:icb_ind+1]
                rs_ic = rs[:icb_ind+1]
                drs_ic = np.diff(rs_ic)
                alpha_ic = alpha[:icb_ind+1]
                Sk_ic = Sk[:icb_ind+1]
                
                #Zero Flux BC
                Tnext[0] = Tk_ic[0] + 2*alpha_ic[0] * (dt / drs_ic[0]**2) * (Tk_ic[1] - Tk_ic[0]) + Sk_ic[0] * dt   


                Tnext[1:icb_ind] = Tk_ic[1:-1]+alpha_ic[1:-1]*(dt/drs_ic[1:]**2)*(Tk_ic[2:]-2*Tk_ic[1:-1] + \
                             Tk_ic[0:-2]) + alpha_ic[1:-1]*(dt/(rs_ic[1:-1]*drs_ic[1:]))*(Tk_ic[2:] - Tk_ic[0:-2]) + \
                             dt * Sk_ic[1:-1]
    
                Tnext[icb_ind] = liquidus 
                
        
        # Set Up Stuff for Crust
        Tk_crust = Tk[crust_inds]
        rs_crust = rs[crust_inds]
        drs_crust = np.diff(rs_crust)
        alpha_crust = alpha[crust_inds]
        Sk_crust = Sk[crust_inds]
        
        q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind]      
        
        if len(crust_inds) > 1:
            Tnext[crust_inds[0]] = Tk_crust[0] + 2*alpha_crust[0] * (dt / drs_crust[0]**2) * \
                    (Tk_crust[1] - Tk_crust[0] - q_crust*drs_crust[0]) + 2 * alpha_crust[0] * (dt / R_oc) * \
                    q_crust + Sk_crust[0] * dt
            

            Tnext[crust_inds[1:-1]] = Tk_crust[1:-1] + alpha_crust[1:-1]*(dt/drs_crust[1:]**2) * \
                            (Tk_crust[2:]-2*Tk_crust[1:-1] + Tk_crust[0:-2]) + alpha_crust[1:-1] * \
                            (dt/(rs_crust[1:-1]*drs_crust[1:])) * (Tk_crust[2:] - Tk_crust[0:-2]) + \
                            dt * Sk_crust[1:-1]

        #constant surface temp
        Tnext[-1] = 137
        
        if len(new_inds) == 0:
            print('fully solidified {} Myr'.format(t/(1e6*365*86400)))
            Ts.append(Tnext)
            time.append(t)
            sulfur.append(sulf)
            next_solid[:] = 1. #the last cell is only ~99% solid
            fs.append(next_solid)
            if eutectic == False:
                next_alpha[sol_ind] = k0_solid / (rho_Fe*Cp)
            else:
                next_alpha[sol_ind] = k0_eu / (rho_eu*Cp_eu)
            alphas.append(next_alpha)
            reached = dc(k)
            t += dt
            if full:
                break
            else:
                return np.asarray(Ts), np.asarray(time), np.asarray(sulfur), np.asarray(fs), np.asarray(alphas)
        
        
        if sulf == 0:
            new_sulf = 0        
        elif sulf < 0.31 and np.nanmin(new_inds) > np.nanmin(fluid_inds):
            M0 = 4/3*np.pi*(rs[np.nanmax(fluid_inds)+1]**3 - rs[np.nanmin(fluid_inds)]**3)*rho_mix(sulf)
            dM = 4/3*np.pi*rho_Fe*(rs[np.nanmax(fluid_inds)+1]**3-rs[np.nanmax(fluid_inds)]**3) #1 cell solidified
            rnew = rs[np.nanmax(new_inds)+1]
            sol = root(get_newS, sulf, args=((M0-dM)/(4/3*np.pi*(rnew**3-rs[np.nanmin(new_inds)]**3))))
            new_sulf = sol.x[0]
        elif sulf >= 0.31:
            new_sulf = 0.31
        else:
            new_sulf = dc(sulf)
            
        if new_sulf > sulf and new_sulf >= 0.31:
            print('eutectic at {} Myr'.format(t/(1e6*365*86400)))
            
        if new_sulf >= 0.31:
            new_sulf = 0.31
            eutectic = True
        
        sulf = dc(new_sulf)
        fluid_inds = dc(new_inds)
        alpha = dc(next_alpha)
        liquidus = liquid(sulf)

        Ts.append(Tnext)
        time.append(t)
        sulfur.append(sulf)
        fs.append(next_solid)
        alphas.append(alpha)

        if t > tmax*3.154e+7:
            print('Reached tmax')
            return np.asarray(Ts), np.asarray(time), np.asarray(sulfur), np.asarray(fs), np.asarray(alphas)

        #change to lower resolution if possible
        limits = np.diff(rs)**2 / (2*alpha[:-1])
        dt = np.nanmin(limits) / 50 #take 1/50th of stability limit as timestep

        t += dt
    
#     NEW LOOP FOR FULL CONDUCTION
    for k in range(reached+1,int(M)+1): #timesteps
        a = rs[0]
        drs = np.diff(rs)
        Tk = Ts[k-1]
        alpha = dc(alphas[k-1])
        Tnext = np.zeros(np.shape(Tk))
        next_solid = dc(fs[k-1])

        ocb_ind = np.nanmin(crust_inds)
        R_oc = rs[ocb_ind]
            
        q_oc = -k0_solid*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind] #negative
        q_crust = k0_rock*(Tk[ocb_ind] - Tk[ocb_ind-1])/ drs[ocb_ind]  

        Tk_ic = Tk[:ocb_ind+1]
        rs_ic = rs[:ocb_ind+1]
        drs_ic = np.diff(rs_ic)
        alpha_ic = alpha[:ocb_ind+1]
        Sk_ic = Sk[:ocb_ind+1]

        #Zero Flux BC
        Tnext[0] = Tk_ic[0] + 2*alpha_ic[0] * (dt / drs_ic[0]**2) * (Tk_ic[1] - Tk_ic[0]) + Sk_ic[0] * dt   


        Tnext[1:ocb_ind] = Tk_ic[1:-1]+alpha_ic[1:-1]*(dt/drs_ic[1:]**2)*(Tk_ic[2:]-2*Tk_ic[1:-1] + \
                     Tk_ic[0:-2]) + alpha_ic[1:-1]*(dt/(rs_ic[1:-1]*drs_ic[1:]))*(Tk_ic[2:] - Tk_ic[0:-2]) + \
                     dt * Sk_ic[1:-1]
        
        # Set Up Stuff for Crust
        Tk_crust = Tk[crust_inds]
        rs_crust = rs[crust_inds]
        drs_crust = np.diff(rs_crust)
        alpha_crust = alpha[crust_inds]
        Sk_crust = Sk[crust_inds]
               
        
        if len(crust_inds) > 1:
            Tnext[crust_inds[0]] = Tk_crust[0] + 2*alpha_crust[0] * (dt / drs_crust[0]**2) * \
                    (Tk_crust[1] - Tk_crust[0] - q_crust*drs_crust[0]) + 2 * alpha_crust[0] * (dt / R_oc) * \
                    q_crust + Sk_crust[0] * dt
            

            Tnext[crust_inds[1:-1]] = Tk_crust[1:-1] + alpha_crust[1:-1]*(dt/drs_crust[1:]**2) * \
                            (Tk_crust[2:]-2*Tk_crust[1:-1] + Tk_crust[0:-2]) + alpha_crust[1:-1] * \
                            (dt/(rs_crust[1:-1]*drs_crust[1:])) * (Tk_crust[2:] - Tk_crust[0:-2]) + \
                            dt * Sk_crust[1:-1]

        #constant surface temp
        Tnext[-1] = 137
        
        Ts.append(Tnext)
        time.append(t)
        sulfur.append(sulf)
        fs.append(next_solid)
        alphas.append(alpha)

        if t > tmax*3.154e+7:
            print('Reached tmax')
            return np.asarray(Ts), np.asarray(time), np.asarray(sulfur), np.asarray(fs), np.asarray(alphas)

        #change to lower resolution if possible
        limits = np.diff(rs)**2 / (2*alpha[:-1])
        dt = np.nanmin(limits) / 5 #take 1/5th of stability limit as timestep

        t += dt
    
    return np.asarray(Ts), np.asarray(time), np.asarray(sulfur), np.asarray(fs), np.asarray(alphas)

# Radial Change Calculations

In [6]:
def contraction_inward(model, shape):
    dr = np.nanmean(np.diff(shape)) #we're using uniform dr
    mantle = np.where(model['f_solid'][0,:] > 0.9)
    
    model_xr = xr.Dataset(data_vars=dict(temperature=(['r', 'time'], model['temps'].T),
                                 sulfur=(['time'], model['sulfur']),
                                 f_solid=(['r', 'time'], model['f_solid'].T),
                                 conduct=(['r', 'time'], model['conducts'].T)
                                ),
                  coords={'r':shape, 
                          'time':model['times']},
                  attrs=dict(description='Inward Solidification, {} km Mantle, {} wt% Sulfur'.format(int((np.nanmax(shape)-np.nanmin(shape[mantle]))/1e3), 
                                                                                                   int(round(model['sulfur'][0]*100)))),
                 )
    
    print(model_xr.description)
    expansion = np.full(np.shape(shape), exp_Fe) #expansivity of iron 
    expansion[mantle] = exp_rock #expansivity of rock
    
    thermal = 1/(np.nanmax(shape))**2 * np.sum(np.diff(model['temps'], axis=0)*expansion*shape**2*dr, axis=1)
    
    cmb = shape[mantle[0][0]]
    
    outer = model_xr.where((model_xr.conduct == k0_solid /(rho_Fe*Cp)) | (model_xr.conduct == k0_eu / (rho_eu*Cp_eu)))
    outer['temperature'][-1,:] = 10000.
    mod = (outer.r * outer.time) + outer.temperature
    icb_ind = np.nanargmin(mod, axis=0)
    
    core = model_xr.where((model_xr.r < cmb) & (model_xr.f_solid < 1.))
    core['f_solid'][0,:] = 0.
    bound = core['f_solid'].max('r')

    icb = outer.r[icb_ind].values + dr*(1-bound)
    icb[np.where(icb_ind==len(outer.r)-1)] = cmb
    
    dV_core = (V_gamma-V_liq)/V_gamma * 4/3 * np.pi * (cmb**3 - (icb)**3)
    phase1 = cmb - np.cbrt((cmb)**3 - (dV_core * 3 / (4*np.pi)))
    
    secondary = model_xr.where((model_xr.temperature <= 1183) & (model_xr.r < cmb))
    secondary['temperature'][-1,:] = 10000.
    mod2 = (secondary.r * secondary.time) + secondary.temperature
    alph_ind = np.nanargmin(mod2, axis=0)
    alpha_boundary = secondary.r[alph_ind].values
    alpha_boundary[np.where(alph_ind==len(secondary.r)-1)] = cmb
    
    eutectic = model_xr.where((model_xr.conduct == k0_eu / (rho_eu*Cp_eu)))
    eutectic['temperature'][0,:] = 10000.
    mod = ((eutectic.r) * eutectic.time) + eutectic.temperature
    eu_ind = np.nanargmax(mod, axis=0)
    r_eu = eutectic.r[eu_ind].values
    
    dV2 = np.full(np.shape(alpha_boundary),0)
    try:
        reg1 = np.where(alpha_boundary > r_eu)
        reg2 = np.nanmax(reg1)+1
        dV2[reg1] = (V_alpha-V_gamma)/V_alpha * 4/3 * np.pi * (cmb**3 - (alpha_boundary[reg1])**3)
        dV2[reg2:] = (V_alpha-V_gamma)/V_alpha * 4/3 * np.pi * ((cmb**3 - r_eu[reg2:]**3) + \
                                                            (r_eu[reg2:]**3 - (alpha_boundary[reg2:])**3)*Fe_perV)
    except ValueError:
        dV2 = (V_alpha-V_gamma)/V_alpha * 4/3 * np.pi * ((cmb**3 - r_eu**3) + \
                                                            (r_eu**3 - (alpha_boundary)**3)*Fe_perV)
    
    phase2 = cmb - np.cbrt((cmb)**3 - dV2 * 3 / (4*np.pi))
    
    print()
    return thermal, phase1, icb, phase2, alpha_boundary


def contraction_outward(model, shape):
    dr = np.nanmean(np.diff(shape)) #we're using uniform dr
    mantle = np.where(model['f_solid'][0,:] > 0.9)
    
    model_xr = xr.Dataset(data_vars=dict(temperature=(['r', 'time'], model['temps'].T),
                             sulfur=(['time'], model['sulfur']),
                             f_solid=(['r', 'time'], model['f_solid'].T),
                             conduct=(['r', 'time'], model['conducts'].T)
                            ),
              coords={'r':shape, 
                      'time':model['times']},
              attrs=dict(description='Outward Solidification, {} km Mantle, {} wt% Sulfur'.format(int((np.nanmax(shape)-np.nanmin(shape[mantle]))/1e3), 
                                                                                               int(round(model['sulfur'][0]*100)))),
             )
    
    print(model_xr.description)
    expansion = np.full(np.shape(shape), exp_Fe) #expansivity of iron
    expansion[mantle] = exp_rock #expansivity of rock
    
    thermal = 1/(np.nanmax(shape))**2 * np.sum(np.diff(model['temps'], axis=0)*expansion*shape**2*dr, axis=1)
    
    cmb = shape[mantle[0][0]]
    
    outer = model_xr.where((model_xr.conduct == k0_solid /(rho_Fe*Cp)) | (model_xr.conduct == k0_eu / (rho_eu*Cp_eu)))
    outer['temperature'][0,:] = 10000.
    mod = (outer.r * outer.time) + outer.temperature
    icb_ind = np.nanargmax(mod, axis=0)

    core = model_xr.where((model_xr.r < cmb) & (model_xr.f_solid < 1.))
    core['f_solid'][0,:] = 0.
    bound = core['f_solid'].max('r')

    icb = outer.r[icb_ind].values + dr*bound
    
    dV_core1 = (V_gamma-V_liq)/V_gamma * 4/3 * np.pi * ((icb)**3)
    phase1 = icb - np.cbrt((icb)**3 - (dV_core1 * 3 / (4*np.pi)))
    
    secondary = model_xr.where((model_xr.temperature <= 1183) & (model_xr.r < cmb))
    secondary['temperature'][-1,:] = 10000.
    mod2 = (secondary.r * secondary.time) + secondary.temperature
    alph_ind = np.nanargmin(mod2, axis=0)
    alpha_boundary = secondary.r[alph_ind].values
    alpha_boundary[np.where(alph_ind==len(secondary.r)-1)] = cmb
    
    eutectic = model_xr.where((model_xr.conduct == k0_eu / (rho_eu*Cp_eu)))
    eutectic['temperature'][mantle[0][0],:] = 10000.
    mod = ((cmb-eutectic.r) * eutectic.time) + eutectic.temperature
    eu_ind = np.nanargmax(mod, axis=0)
    r_eu = eutectic.r[eu_ind].values
    r_eu[np.where(eu_ind==0)] = cmb
    
    dV2 = np.full(np.shape(alpha_boundary),0)
    try:
        reg1 = np.where(alpha_boundary > r_eu)
        reg2 = np.nanmax(reg1)+1
        dV2[reg1] = (V_alpha-V_gamma)/V_alpha * 4/3 * np.pi * (cmb**3 - (alpha_boundary[reg1])**3)*Fe_perV
        dV2[reg2:] = (V_alpha-V_gamma)/V_alpha * 4/3 * np.pi * ((cmb**3 - r_eu[reg2:]**3)*Fe_perV + \
                                                                (r_eu[reg2:]**3 - (alpha_boundary[reg2:])**3))
    except ValueError:
        dV2 = (V_alpha-V_gamma)/V_alpha * 4/3 * np.pi * ((cmb**3 - r_eu**3)*Fe_perV + \
                                                                (r_eu**3 - (alpha_boundary)**3))
        
    phase2 = cmb - np.cbrt((cmb)**3 - dV2 * 3 / (4*np.pi))
    
    print()
    return thermal, phase1, icb, phase2, alpha_boundary

def contraction_porous(model):
    shape = model['shapes']
    
    expansion = np.full(np.shape(shape[0,:]), exp_Fe) #expansivity of iron
    
    thermal = 1/(np.nanmax(shape, axis=1)[1:])**2 * np.sum(np.diff(model['temps'], axis=0)[:,1:]* \
                                    expansion[1:]*shape[1:,1:]**2*np.diff(shape, axis=1)[1:,:], axis=1)
    
    removal = shape[1:, -1] - shape[0, -1]
    
    return thermal, removal

# MOI Calculations

In [7]:
def calc_MOI(model, shape):
    dr = np.nanmean(np.diff(shape)) #we're using uniform dr
    mantle = np.where(model['f_solid'][0,:] > 0.9)
    eutectic = np.where(model['conducts'][-1,:] == k0_eu / (rho_eu*Cp_eu))
    cmb = shape[mantle[0][0]]
    R = shape[-1]
    try:
        if eutectic[0][1] == 1:#np.nanmin(eutectic) == 0:
            r_eu = np.nanmax(shape[eutectic])
            I_eu = 8/15*rho_eu*np.pi*(r_eu**5)
            I_core = 8/15*rho_Fe*np.pi*(cmb**5 - r_eu**5)
            
            print('inward', r_eu/1e3, cmb/1e3, R/1e3) #eutectic at center
            
        else:
            icb = shape[eutectic[0][1]]
            print('Outward', icb/1e3, cmb/1e3, R/1e3) #eutectic above solid Fe inner core
            
            I_core = 8/15*rho_Fe*np.pi*(icb**5)
            I_eu = 8/15*rho_eu*np.pi*(cmb**5 - icb**5)
            
    except (ValueError, IndexError):
        print('no sulfur')
        r_eu = 0.
        I_eu = 8/15*rho_eu*np.pi*(r_eu**5)
        I_core = 8/15*rho_Fe*np.pi*(cmb**5 - r_eu**5)    
    
    I_mant = 8/15*rho_rock*np.pi*(R**5 - cmb**5)
    
    return (I_eu+I_core+I_mant), (I_eu+I_core+I_mant)/(2.3e19*R**2)

def calc_MOI_obl(model, shape):
    dr = np.nanmean(np.diff(shape)) #we're using uniform dr
    mantle = np.where(model['f_solid'][0,:] > 0.9)
    eutectic = np.where(model['conducts'][-1,:] == k0_eu / (rho_eu*Cp_eu))
    cmb = shape[mantle[0][0]]
    R = shape[-1]
    aspect = 171/278 #1/Psyche's aspect ratio (Shepard+ 2021)
    a = np.cbrt(R**3/aspect)
    C = a*aspect 
    thick_m = R-cmb
    c_cmb = C-thick_m #keep uniform mantle thickness
    a_cmb = a-thick_m
    
    try:
        if eutectic[0][1] == 1:
            r_eu = np.nanmax(shape[eutectic])
            thick_Fe = cmb-r_eu
            c_eu = c_cmb - thick_Fe
            a_eu = a_cmb - thick_Fe
            
            I_eu = 8/15*rho_eu*np.pi*(a_eu**4*c_eu)
            I_core = 8/15*rho_Fe*np.pi*(a_cmb**4*c_cmb - a_eu**4*c_eu)
            
            M = 4/3*np.pi*(rho_eu*(a_eu**2*c_eu)+rho_Fe*(a_cmb**2*c_cmb-a_eu**2*c_eu)+rho_rock*(a**2*C-a_cmb**2*c_cmb))
            
            print('inward', a_eu/1e3, a_cmb/1e3, a/1e3) #eutectic at center
            
        else:
            icb = shape[eutectic[0][1]]
            thick_eu = cmb - icb
            c_icb = c_cmb - thick_eu
            a_icb = a_cmb - thick_eu
            print('Outward', a_icb/1e3, a_cmb/1e3, a/1e3) #eutectic above solid Fe inner core
            
            M = 4/3*np.pi*(rho_Fe*(a_icb**2*c_icb)+rho_eu*(a_cmb**2*c_cmb-a_icb**2*c_icb)+rho_rock*(a**2*C-a_cmb**2*c_cmb))
            
            I_core = 8/15*rho_Fe*np.pi*(a_icb**4*c_icb)
            I_eu = 8/15*rho_eu*np.pi*(a_cmb**4*c_cmb - a_icb**4*c_icb)
            
    except (ValueError, IndexError):
        print('no sulfur')
        r_eu = 0.
        c_eu = 0.
        I_eu = 8/15*rho_eu*np.pi*(r_eu**4*c_eu)
        I_core = 8/15*rho_Fe*np.pi*(a_cmb**4*c_cmb - r_eu**4*c_eu)
        
        M = 4/3*np.pi*(rho_Fe*(a_cmb**2*c_cmb)+rho_rock*(a**2*C-a_cmb**2*c_cmb))
    
    I_mant = 8/15*rho_rock*np.pi*(a**4*C - a_cmb**4*c_cmb)
    
   
    
    return (I_eu+I_core+I_mant), (I_eu+I_core+I_mant)/(M*a**2) #here a > C

def calc_MOI_obl2(model, shape): #spherical core
    dr = np.nanmean(np.diff(shape)) #we're using uniform dr
    mantle = np.where(model['f_solid'][0,:] > 0.9)
    eutectic = np.where(model['conducts'][-1,:] == k0_eu / (rho_eu*Cp_eu))
    cmb = shape[mantle[0][0]]
    R = shape[-1]
    aspect = 171/278 #1/Psyche's aspect ratio (Shepard+ 2021)
    a = np.cbrt(R**3/aspect)
    C = a*aspect 
    
    if cmb >= C:
        print('uh-oh (cmb > C)')
        cmb = C
    
    try:
        if eutectic[0][1] == 1:
            r_eu = np.nanmax(shape[eutectic])
            
            I_eu = 8/15*rho_eu*np.pi*(r_eu**5)
            I_core = 8/15*rho_Fe*np.pi*(cmb**5 - r_eu**5)
            
            M = 4/3*np.pi*(rho_eu*(r_eu**3)+rho_Fe*(cmb**3-r_eu**3)+rho_rock*(a**2*C-cmb**3))
            
            print('inward', r_eu/1e3, cmb/1e3, a/1e3) #eutectic at center
            
        else:
            icb = shape[eutectic[0][1]]

            print('Outward', icb/1e3, cmb/1e3, a/1e3) #eutectic above solid Fe inner core
            
            M = 4/3*np.pi*(rho_Fe*(icb**3)+rho_eu*(cmb**3-icb**3)+rho_rock*(a**2*C-cmb**3))
            
            I_core = 8/15*rho_Fe*np.pi*(icb**5)
            I_eu = 8/15*rho_eu*np.pi*(cmb**5 - icb**5)
            
    except (ValueError, IndexError):
        print('no sulfur')
        r_eu = 0.
        c_eu = 0.
        I_eu = 8/15*rho_eu*np.pi*(r_eu**5)
        I_core = 8/15*rho_Fe*np.pi*(cmb**5 - r_eu**5)
        
        M = 4/3*np.pi*(rho_Fe*(cmb**3)+rho_rock*(a**2*C-cmb**3))
    
    I_mant = 8/15*rho_rock*np.pi*(a**4*C - cmb**5)
    
   
    
    return (I_eu+I_core+I_mant), (I_eu+I_core+I_mant)/(M*a**2) #here a > C

# Example Psyche Model

In [8]:
sulf_init = 0.01 #wt fraction
mant_thick = 1 #km
M = 2.3e19 #mass of body
tmax = 1e5 #in years

sol = root(body_size, 110e3, args=(M, sulf_init, mant_thick*1e3))
rmax = int(np.round(sol.x[0]/1e3)) #in km, matches measured mass
N = rmax*2 #want 1/2 km res

dt = (tmax * 3.154e+7) / (tmax/1000) #want 1000 yr res
dr = (rmax * 1e3) / N

print(mant_thick, 'km and', sulf_init*100, 'wt%')
print(rmax, dr)

core = np.arange(0,N-(2*mant_thick))
mantle = np.arange(N-(2*mant_thick),N+1)
T0 = np.full((N+1), 1820)
T0[mantle] = 1300
T0[-1] = 137

start = time.time()
psyche, times, sulfurs, f_sols, alphs = inward(np.arange(0,rmax*1e3+dr, dr), T0, tmax, mantle, core, t_acc=0, 
                                    sulf=sulf_init, phi_mant=0, full=True)
print("--- %s minutes ---" % ((time.time() - start)/60))

1 km and 1.0 wt%
93 500.0
Initial Sulfur Content: 1.0 wt%
solidification start 0.008228765339611872 Myr
Reached tmax
--- 0.019639201958974204 minutes ---


In [9]:
path = 'Inward{}kmMantle{}wtSulfur.npz'.format(mant_thick, int(sulf_init*100))

np.savez_compressed(path, times=times.astype(np.float32), temps=psyche.astype(np.float32), 
                    sulfur=sulfurs.astype(np.float32), f_solid=f_sols.astype(np.float32), 
                    conducts=alphs.astype(np.float32))

In [10]:
inward1km_1wt = np.load('Inward1kmMantle1wtSulfur.npz')
rmax1km_1wt = (np.shape(inward1km_1wt['temps'])[1]-1)/2

In [11]:
#thermal dr, phase change dr, inner core boundary, gamma to alpha phase change dr, second Fe phase boundary
t_i1_1wt, p_i1_1wt, icb_i1_1wt, p2_i1_1wt, b_i1_1wt = contraction_inward(inward1km_1wt, 
                                                                            np.arange(0,rmax1km_1wt*1e3+dr, dr))

#full radial change
full_i1_1wt = np.cumsum(t_i1_1wt)+p_i1_1wt[:-1]+p2_i1_1wt[:-1]

Inward Solidification, 1 km Mantle, 1 wt% Sulfur



In [12]:
#moment of inertia, moment of inertia factor
MOI, fraction = calc_MOI(inward1km_1wt, np.arange(0,rmax1km_1wt*1e3+dr, dr))
print(fraction)

no sulfur
0.4265217578035331
